# Extra Trees Model Training (from scraped GitHub code)

This notebook trains an **Extra Trees** model on the metrics dataset 


## 1) Load Lib

In [49]:
# Step 1: Imports
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import accuracy_score, classification_report
import pickle
from pathlib import Path
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')

## 2) Load dataset

In [50]:
df = pd.read_csv('../data/processed/dataset_processed.csv')
df.head()

,average_cyclomatic_complexity,average_methods_per_class,avg_line_length,boolean_expression_avg_terms,call_graph_density,classes,classes_with_inheritance,comment_code_mismatch_score,comment_lines,comment_percentage,commit_bursts,coupled_file_changes,cross_file_call_edges,decision_density,documentation_coverage,external_vs_internal_field_access_ratio,file_age_days,file_path,functions,global_usages_total,globals_declared,god_class_proxies,halstead_difficulty,halstead_effort,halstead_estimated_bugs,halstead_volume,indentation_irregularity,inter_file_coupling,large_parameter_list_indicator,lazy_class_indicator,lines_added,lines_deleted,lines_of_code,long_method_indicator,max_cyclomatic_ratio,max_intra_file_call_depth,max_line_length,max_lines_per_function,max_nesting_level,mean_cyclomatic_ratio,mean_lines_per_function,mean_param_entropy,methods,nesting_variance,num_authors,pep8_examples,pep8_violations,percent_lines_over_80,semantic_todo_density,source_lines,test_files_found,test_function_count,test_lines,test_to_source_ratio,todo_fixme_count,todo_fixme_semantic_density,total_imports,unit_test_presence,vcs_top_coupled,y_FeatureEnvy,y_FormattingIssues,y_GlobalStateAbuse,y_GodClass,y_LargeParameterList,y_LazyClass,y_LongMethod,y_MisleadingComments,y_PoorDocumentation,y_SpaghettiCode,y_UnstableModule,y_UntestedCode,y_any_smell
0,0.360756,0.0,0.394464,0.0,0.333333,0.00000,0.000,0.0,0.125000,0.167,0.0,{},"[{'caller': 'sort_blocks', 'module': 'read_me'...",0.623314,0.0,0.033846,0.0,awesome-python\sort.py,0.025974,0.00000,0.000000,[],0.101876,0.010526,0.026215,0.026210,"{'indent_stddev': 3.77, 'irregularity_score': ...",0.105263,False,False,0.011375,0.0,0.011375,False,0.172667,0.125,0.115646,0.005102,0.3,0.254697,0.118520,0.000000,0.0,0.12288,0.0,"['line 14: length 80', 'line 38: length 102']",0.002972,0.035704,0.0,0.006111,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,[],1,0,0,0,0,0,0,0,1,0,0,1,1
1,0.166203,0.0,0.369089,0.0,0.000000,0.00000,0.000,0.0,0.017857,0.095,0.0,{},"[{'caller': 'pdm_build_initialize', 'module': ...",0.326629,0.0,0.009231,0.0,fastapi\pdm_build.py,0.012987,0.00625,0.038462,[],0.000000,0.000000,0.000000,0.000000,"{'indent_stddev': 2.78, 'irregularity_score': ...",0.017544,False,False,0.002741,0.0,0.002741,False,0.166667,0.000,0.083900,0.001749,0.1,0.260960,0.045879,0.613327,0.0,0.00000,0.0,[],0.000000,0.000000,0.0,0.001944,1.0,1.0,1.0,0.071766,0.0,0.0,0.038961,0.0,[],0,0,0,0,1,0,0,0,1,0,0,0,1
2,0.110617,0.0,0.242215,0.0,0.000000,0.04878,0.050,0.0,0.000000,0.000,0.0,{},"[{'caller': 'read_item', 'module': 'app'}]",0.163200,0.0,0.003077,0.0,fastapi\docs_src\additional_responses\tutorial...,0.000000,0.00625,0.038462,[],0.000000,0.000000,0.000000,0.000000,"{'indent_stddev': 2.51, 'irregularity_score': ...",0.017544,False,False,0.003015,0.0,0.003015,False,0.333333,0.000,0.097506,0.000583,0.1,0.521921,0.015293,0.613327,0.0,0.00000,0.0,['line 18: length 86'],0.001486,0.130513,0.0,0.001944,0.0,0.0,0.0,0.000000,0.0,0.0,0.038961,0.0,[],0,1,0,0,0,1,0,0,1,0,0,0,1
3,0.110617,0.0,0.226067,0.0,0.000000,0.02439,0.025,0.0,0.000000,0.000,0.0,{},"[{'caller': 'read_item', 'module': 'app'}]",0.104000,0.0,0.003077,0.0,fastapi\docs_src\additional_responses\tutorial...,0.000000,0.00625,0.038462,[],0.021992,0.000387,0.004495,0.004494,"{'indent_stddev': 3.81, 'irregularity_score': ...",0.017544,False,False,0.003837,0.0,0.003837,False,0.266667,0.000,0.072562,0.000729,0.1,0.417537,0.019116,0.643482,0.0,0.00000,0.0,[],0.000000,0.000000,0.0,0.003056,0.0,0.0,0.0,0.000000,0.0,0.0,0.038961,0.0,[],0,0,0,0,0,1,0,0,1,0,0,0,1
4,0.110617,0.0,0.222607,0.0,0.000000,0.02439,0.025,0.0,0.000000,0.000,0.0,{},"[{'caller': 'read_item', 'module': 'app'}]",0.099429,0.0,0.003077,0.0,fastapi\docs_src\additional_responses\tutorial...,0.000000,0.00625,0.038462,[],0.000000,0.000000,0.000000,0.000000,"{'indent_stddev': 3.82, 'irregularity_score': ...",0.017544,False,False,0.004111,0.0,0.004111,False,0.266667,0.000,0.073696,0.000729,0.1,0.417537,0.019116,0.643482,0.0,0.00000

In [51]:
# Features = everything except target columns that start with "y_"
target_cols = [c for c in df.columns if c.startswith("y_")]
if not target_cols:
    raise ValueError("No columns found starting with 'y_'")

X = df.drop(columns=target_cols)
Y = df[target_cols]  # DataFrame containing all y_ targets

# Fix: make sure X is numeric (scikit-learn can't train on strings like "{}")
X = X.replace("{}", pd.NA)
for col in X.columns:
    if X[col].dtype == "object":
        X[col] = pd.to_numeric(X[col], errors="coerce")
X = X.fillna(0)

print("Targets:", target_cols)
print("X shape:", X.shape)

Targets: ['y_FeatureEnvy', 'y_FormattingIssues', 'y_GlobalStateAbuse', 'y_GodClass', 'y_LargeParameterList', 'y_LazyClass', 'y_LongMethod', 'y_MisleadingComments', 'y_PoorDocumentation', 'y_SpaghettiCode', 'y_UnstableModule', 'y_UntestedCode', 'y_any_smell']
X shape: (3088, 59)


## 3) Split the data

In [52]:
# Pick ONE y_ column to train on (keeps notebook simple)
TARGET = target_cols[0]  # change to any y_ column you want
y = Y[TARGET].copy()

# If y is continuous numeric, convert to 0/1 classes (needed for classifier)
if pd.api.types.is_numeric_dtype(y) and y.nunique() > 2:
    y = (y >= y.median()).astype(int)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y if y.nunique() > 1 else None
)

print('Training target:', TARGET)
y.value_counts(dropna=False).head()

Training target: y_FeatureEnvy


y_FeatureEnvy
0    1981
1    1107
Name: count, dtype: int64

## 4) Train Extra Trees

In [53]:
# Step 5: Tune parameters (simple) + train best model (more anti-overfitting)
param_dist = {
    "n_estimators": [200, 400, 800],
    "max_depth": [10, 20, 30],
    "min_samples_split": [5, 10, 20],
    "min_samples_leaf": [2, 4, 8],
    "max_features": ["sqrt", "log2", 0.5],

}

base_model = ExtraTreesClassifier(random_state=42, n_jobs=-1)

# Step 6: Randomized Search CV
search = RandomizedSearchCV(
    estimator=base_model,
    param_distributions=param_dist,
    n_iter=30,
    scoring="accuracy",
    cv=5,
    random_state=42,
    n_jobs=-1,
    verbose=1,
    error_score="raise",
)

search.fit(X_train, y_train)

print("Best CV accuracy:", search.best_score_)
print("Best params:", search.best_params_)

et_model = search.best_estimator_


Fitting 5 folds for each of 30 candidates, totalling 150 fits
Best CV accuracy: 0.9814889658711831
Best params: {'n_estimators': 200, 'min_samples_split': 20, 'min_samples_leaf': 4, 'max_features': 0.5, 'max_depth': 20}


## 5) Evaluate

In [54]:
# Step 6: Make predictions
y_pred = et_model.predict(X_test)

# Step 7: Evaluate
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.9816612729234089

Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.98      0.99       595
           1       0.96      0.98      0.97       332

    accuracy                           0.98       927
   macro avg       0.98      0.98      0.98       927
weighted avg       0.98      0.98      0.98       927



In [55]:

train_acc = et_model.score(X_train, y_train)
test_acc = et_model.score(X_test, y_test)
print("Train accuracy:", train_acc)
print("Test accuracy:", test_acc)
print("Gap:", train_acc - test_acc)

Train accuracy: 0.99537251272559
Test accuracy: 0.9816612729234089
Gap: 0.013711239802181141


In [56]:
def overfitting_score(train_acc, test_acc, cv_mean, cv_std):
    """
    Calculate overfitting indicators
    """
    # Indicator 1: Train-Test gap
    train_test_gap = train_acc - test_acc
    
    # Indicator 2: CV-Test gap
    cv_test_gap = cv_mean - test_acc
    
    # Indicator 3: Relative overfitting
    relative_overfit = (train_acc - test_acc) / train_acc * 100
    
    return {
        'train_test_gap': train_test_gap,
        'cv_test_gap': cv_test_gap,
        'relative_overfit_percent': relative_overfit,
        'cv_std': cv_std
    }

# Extract CV scores from search object
cv_scores = search.cv_results_['mean_test_score']

# Calculate metrics
metrics = overfitting_score(train_acc, test_acc, cv_scores.mean(), cv_scores.std())

print("\n=== Overfitting Analysis ===")
print(f"Train-Test Accuracy Gap: {metrics['train_test_gap']:.4f}")
print(f"CV Mean - Test Gap: {metrics['cv_test_gap']:.4f}")
print(f"Relative Overfitting: {metrics['relative_overfit_percent']:.2f}%")
print(f"CV Standard Deviation: {metrics['cv_std']:.4f}")

# Interpretation
print("\n=== Interpretation ===")
if metrics['train_test_gap'] > 0.05:
    print("⚠️  WARNING: Possible overfitting (gap > 0.05)")
elif metrics['train_test_gap'] > 0.02:
    print("⚠️  Mild overfitting detected")
else:
    print("✅ Good generalization (gap < 0.02)")

if metrics['cv_std'] > 0.05:
    print("⚠️  High variance in CV scores")


=== Overfitting Analysis ===
Train-Test Accuracy Gap: 0.0137
CV Mean - Test Gap: -0.0519
Relative Overfitting: 1.38%
CV Standard Deviation: 0.0409

=== Interpretation ===
✅ Good generalization (gap < 0.02)


## 6) Save model

In [36]:
# Step 8: Save model
from pathlib import Path
import pickle

models_dir = Path("../models")
models_dir.mkdir(parents=True, exist_ok=True)

model_path = models_dir / "extra_trees_classifier.pkl"
with open(model_path, "wb") as f:
    pickle.dump(et_model, f)

print("Extra Trees classifier saved successfully as pickle! ->", model_path)

Extra Trees classifier saved successfully as pickle! -> ../models/extra_trees_classifier.pkl
